[adaptado de [Programa de cursos integrados Aprendizado de máquina](https://www.coursera.org/specializations/machine-learning-introduction) de [Andrew Ng](https://www.coursera.org/instructor/andrewng)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/) ) ]

In [ ]:
# Baixar arquivos adicionais para o laboratório.
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/ml_intro/class_03/Laborat%C3%B3rios/lab_utils_ml_intro_week_3.zip
!unzip -n -q lab_utils_ml_intro_week_3.zip

In [ ]:
# Testar se estamos no Google Colab
try:
  import google.colab
  IN_COLAB = True
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  IN_COLAB = False

# Regularização de Função de Custo e Gradiente

## Objetivos
Neste laboratório, você irá:
- estender as anteriores funções de custo linear e logístico com um termo de regularização.
- execute novamente o exemplo anterior de overfitting com um termo de regularização adicionado.

In [ ]:
!pip install ipympl

In [ ]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from plt_overfit import overfit_example, output
from lab_utils_common import sigmoid
np.set_printoptions(precision=8)

# Adicionando Regularização
<img align="Left" src="./images/C1_W3_LinearGradientRegularized.png"  style=" width:400px; padding: 10px; " >
<img align="Center" src="./images/C1_W3_LogisticGradientRegularized.png"  style=" width:400px; padding: 10px; " >

Os slides acima mostram as funções de custo e gradiente para regressão linear e logística. Observação:
- Custo
     - As funções de custo diferem significativamente entre regressão linear e logística, mas adicionar regularização às equações é a mesma coisa.
- Gradiente
     - As funções de gradiente para regressão linear e logística são muito semelhantes. Eles diferem apenas na implementação de $f_{wb}$.

## Funções de custo com regularização
### Função de custo para regressão linear regularizada

A equação para a regressão linear regularizada da função de custo é:
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2  + \frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2 \tag{1}$$ 
where:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{2} $$ 


Compare isso com a função de custo sem regularização (que você implementou em um laboratório anterior), que tem o formato:

$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 $$ 

A diferença é o termo de regularização,  <span style="color:blue">
    $\frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2$ </span> 
    
A inclusão deste termo incentiva a descida do gradiente para minimizar o tamanho dos parâmetros. Observe que neste exemplo o parâmetro $b$ não está regularizado. Esta é uma prática padrão.

Abaixo está uma implementação das equações (1) e (2). Observe que isso usa um *padrão padrão para este curso*, um `for loop` sobre todos os `m` exemplos.

In [ ]:
def compute_cost_linear_reg(X, y, w, b, lambda_ = 1):
    """
    Calcula o custo de todos os exemplos
    Args:
      X (ndarray (m,n): Dados, m exemplos com n recursos
      y (ndarray (m,)): valores alvo
      w (ndarray (n,)): parâmetros do modelo
      b (scalar)      : parâmetros do modelo
      lambda_ (scalar): Controla a quantidade de regularização
    Returns:
      total_cost (scalar):  cost 
    """

    m  = X.shape[0]
    n  = len(w)
    cost = 0.
    for i in range(m):
        f_wb_i = np.dot(X[i], w) + b                                   #(n,)(n,)=escalar, veja np.dot
        cost = cost + (f_wb_i - y[i])**2                               #escalar             
    cost = cost / (2 * m)                                              #escalar  
 
    reg_cost = 0
    for j in range(n):
        reg_cost += (w[j]**2)                                          #escalar
    reg_cost = (lambda_/(2*m)) * reg_cost                              #escalar
    
    total_cost = cost + reg_cost                                       #escalar
    return total_cost                                                  #escalar

Execute a célula abaixo para vê-la em ação.

In [ ]:
np.random.seed(1)
X_tmp = np.random.rand(5,6)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1]).reshape(-1,)-0.5
b_tmp = 0.5
lambda_tmp = 0.7
cost_tmp = compute_cost_linear_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print("Custo regularizado:", cost_tmp)

**Saída Esperada**:
<table>
  <tr>
    <td> <b>Custo regularizado: </b> 0.07917239320214275 </td>
  </tr>
</table>

### Função de custo para regressão logística regularizada
Para regressão **logística** regularizada, a função de custo tem a forma
$$J(\mathbf{w},b) = \frac{1}{m}  \sum_{i=0}^{m-1} \left[ -y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \right] + \frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2 \tag{3}$$
onde:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = sigmoid(\mathbf{w} \cdot \mathbf{x}^{(i)} + b)  \tag{4} $$ 

Compare isso com a função de custo sem regularização (que você implementou em um laboratório anterior):

$$ J(\mathbf{w},b) = \frac{1}{m}\sum_{i=0}^{m-1} \left[ (-y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right)\right] $$

Como foi o caso da regressão linear acima, a diferença é o termo de regularização, que é <span style="color:blue">
    $\frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2$ </span> 

A inclusão deste termo incentiva a descida do gradiente para minimizar o tamanho dos parâmetros. Observe que neste exemplo o parâmetro $b$ não está regularizado. Esta é uma prática padrão.

In [ ]:
def compute_cost_logistic_reg(X, y, w, b, lambda_ = 1):
    """
    Calcula o custo de todos os exemplos
    Args:
    Args:
      X (ndarray (m,n): Dados, m exemplos com n recursos
      y (ndarray (m,)): valores alvo
      w (ndarray (n,)): parâmetros do modelo
      b (scalar)      : parâmetros do modelo
      lambda_ (scalar): Controla a quantidade de regularização
    Returns:
      total_cost (scalar):  cost 
    """

    m,n  = X.shape
    cost = 0.
    for i in range(m):
        z_i = np.dot(X[i], w) + b                                      #(n,)(n,)=escalar, veja np.dot
        f_wb_i = sigmoid(z_i)                                          #escalar
        cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)      #escalar
             
    cost = cost/m                                                      #escalar

    reg_cost = 0
    for j in range(n):
        reg_cost += (w[j]**2)                                          #escalar
    reg_cost = (lambda_/(2*m)) * reg_cost                              #escalar
    
    total_cost = cost + reg_cost                                       #escalar
    return total_cost                                                  #secalar

Execute a célula abaixo para vê-la em ação.

In [ ]:
np.random.seed(1)
X_tmp = np.random.rand(5,6)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1]).reshape(-1,)-0.5
b_tmp = 0.5
lambda_tmp = 0.7
cost_tmp = compute_cost_logistic_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print("Custo Regularizado:", cost_tmp)

**Saída Esperada**:
<table>
  <tr>
    <td> <b>Custo Regularizado: </b> 0.6850849138741673 </td>
  </tr>
</table>

## Gradiente descendente com regularização
O algoritmo básico para executar a descida gradiente não muda com a regularização, é:
$$\begin{align*}
&\text{repeat until convergence:} \; \lbrace \\
&  \; \; \;w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j := 0..n-1} \\ 
&  \; \; \;  \; \;b = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \\
&\rbrace
\end{align*}$$
Onde cada atualização realiza atualizações simultâneas em $w_j$ para todo $j$.

O que muda com a regularização é o cálculo dos gradientes.

### Calculando o Gradiente com regularização (tanto linear/logística)
O cálculo do gradiente para regressão linear e logística é quase idêntico, diferindo apenas no cálculo de $f_{\mathbf{w}b}$.
$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)}  +  \frac{\lambda}{m} w_j \tag{2} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3} 
\end{align*}$$

* m é o número de exemplos de treinamento no conjunto de dados
* $f_{\mathbf{w},b}(x^{(i)})$ é a previsão do modelo, enquanto $y^{(i)}$ é o alvo

      
* Para um modelo de regressão <span style="color:blue"> **linear** </span>
    $f_{\mathbf{w},b}(x) = \mathbf{w} \cdot \mathbf{x} + b$  
* Para um modelo de regressão <span style="color:blue"> **logistico** </span>
    $z = \mathbf{w} \cdot \mathbf{x} + b$  
    $f_{\mathbf{w},b}(x) = g(z)$  
    onde $g(z)$ é a função sigmoid:  
    $g(z) = \frac{1}{1+e^{-z}}$   
    
O termo que adiciona regularização é o $\frac{\lambda}{m} w_j$.

### Função gradiente para regressão linear regularizada

In [ ]:
def compute_gradient_linear_reg(X, y, w, b, lambda_): 
    """
    Calcula o gradiente para regressão linear
    Args:
      X (ndarray (m,n): Dados, m exemplos com n recursos
      y (ndarray (m,)): valores alvo
      w (ndarray (n,)): parâmetros do modelo
      b (scalar)      : parâmetros do modelo
      lambda_ (scalar): Controla a quantidade de regularização
      
    Returns:
      dj_dw (ndarray (n,)): O gradiente do custo em relação ao valor os parâmetros w.
      dj_db (scalar):       O gradiente do custo em relação ao valor os parâmetros b.
    """
    m,n = X.shape           #(número de exemplos, número de recursos)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]                 
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]               
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m   
    
    for j in range(n):
        dj_dw[j] = dj_dw[j] + (lambda_/m) * w[j]

    return dj_db, dj_dw

Execute a célula abaixo para vê-la em ação.

In [ ]:
np.random.seed(1)
X_tmp = np.random.rand(5,3)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1])
b_tmp = 0.5
lambda_tmp = 0.7
dj_db_tmp, dj_dw_tmp =  compute_gradient_linear_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print(f"dj_db: {dj_db_tmp}", )
print(f"dj_dw regularizado:\n {dj_dw_tmp.tolist()}", )

### Função gradiente para regressão logística regularizada

In [ ]:
def compute_gradient_logistic_reg(X, y, w, b, lambda_): 
    """

    Calcula o gradiente para regressão logística
    Args:
      X (ndarray (m,n): Dados, m exemplos com n recursos
      y (ndarray (m,)): valores alvo
      w (ndarray (n,)): parâmetros do modelo
      b (scalar)      : parâmetros do modelo
      lambda_ (scalar): Controla a quantidade de regularização
      
    Returns:
      dj_dw (ndarray (n,)): O gradiente do custo em relação ao valor os parâmetros w.
      dj_db (scalar):       O gradiente do custo em relação ao valor os parâmetros b.    
    Computes the gradient for linear regression 
    """
    m,n = X.shape
    dj_dw = np.zeros((n,))                            #(n,)
    dj_db = 0.0                                       #scalar

    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i],w) + b)          #(n,)(n,)=scalar
        err_i  = f_wb_i  - y[i]                       #scalar
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]      #scalar
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m                                   #(n,)
    dj_db = dj_db/m                                   #scalar

    for j in range(n):
        dj_dw[j] = dj_dw[j] + (lambda_/m) * w[j]

    return dj_db, dj_dw  


Execute a célula abaixo para vê-la em ação.

In [ ]:
np.random.seed(1)
X_tmp = np.random.rand(5,3)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1])
b_tmp = 0.5
lambda_tmp = 0.7
dj_db_tmp, dj_dw_tmp =  compute_gradient_logistic_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print(f"dj_db: {dj_db_tmp}", )
print(f"dj_dw regularizado:\n {dj_dw_tmp.tolist()}", )

**Expected Output**
```
dj_db: 0.341798994972791
Regularized dj_dw:
 [0.17380012933994293, 0.32007507881566943, 0.10776313396851499]
 ```

## Execute novamente o exemplo de overfitting

In [ ]:
plt.close("all")
display(output)
ofit = overfit_example(True)

No gráfico acima, experimente a regularização no exemplo anterior. Em particular:
- Categórico (regressão logística)
     - defina o grau como 6, lambda como 0 (sem regularização), ajuste os dados
     - agora defina lambda como 1 (aumenta a regularização), ajuste os dados, observe a diferença.
- Regressão (regressão linear)
     - tente o mesmo procedimento.

## Parabéns!
Você tem:
- exemplos de rotinas de custo e gradiente com regularização adicionada para regressão linear e logística
- desenvolveu alguma intuição sobre como a regularização pode reduzir o overfitting